In [6]:
import json
from pyowm import OWM
import yaml

## Weather API Example

Load your API Key to initialize the Python Wrapper

Interaction with [Open Waeather Map API](https://openweathermap.org/api). The main endpoints return the current weather, the forecasting for next days and historical weather. As for Twitter, it is necessary to generate an API key to obtain access to the endpoints.

Go to https://openweathermap.org/appid and follow the instructions.

In [8]:
# Personal API Key
with open(r'./weather_API.yaml') as file:
    api_key = yaml.full_load(file)["weather_API"]

In [9]:
# Initialize Weather Manager
owm = OWM(api_key)
mgr = owm.weather_manager()

Get weather given a place name

In [10]:
# Collect Weather at given Location
location = 'London, GB'
obs1 = mgr.weather_at_place(location)
w1 = obs1.weather
print("The weather in",location,"is", w1.detailed_status)

The weather in London, GB is broken clouds


Get weather given a location

In [11]:
# Collect Weather at given Coordinates
lat = 45.5209927
long = 9.2102195
obs2 = mgr.weather_at_coords(lat,long)
w2 = obs2.weather
print("The weather in",location,"is", w2.detailed_status)

The weather in London, GB is clear sky


In [12]:
# Collect the weather around a specific location
i = 0
obs_list = mgr.weather_around_coords(lat, long, limit=8)

for el in obs_list:
    i = i + 1
    print(i,"- The weather in around the specified coordinates is", el.weather.detailed_status)

1 - The weather in around the specified coordinates is clear sky
2 - The weather in around the specified coordinates is clear sky
3 - The weather in around the specified coordinates is clear sky
4 - The weather in around the specified coordinates is clear sky
5 - The weather in around the specified coordinates is clear sky
6 - The weather in around the specified coordinates is clear sky
7 - The weather in around the specified coordinates is clear sky
8 - The weather in around the specified coordinates is clear sky


It is possible to access the fields also using getter methdos.

In [13]:
kelvin_temp = w2.temperature()
celsius_temp = w2.temperature('celsius')
print("Kelvin) Max temp:",kelvin_temp['temp_max'],"- Min temp:",kelvin_temp['temp_min'])
print("Kelvin) Max temp:",celsius_temp['temp_max'],"- Min temp:",celsius_temp['temp_min'])

Kelvin) Max temp: 288.12 - Min temp: 282.92
Kelvin) Max temp: 14.97 - Min temp: 9.77


In [14]:
location2 = 'Tokyo,JP'
obs3 = mgr.weather_at_place(location2)
wind = obs3.weather.wind()
print("The speed of the wind in",location2,"is",wind['speed'],"m/s")

The speed of the wind in Tokyo,JP is 7.2 m/s


Get next week forecast for a specific location

In [15]:
# ISO sunset and sunrise
location2 = 'Milan,IT'
obs4 = mgr.weather_at_place(location2)
weatherIT = obs4.weather
sunrise_unix = weatherIT.sunrise_time()
sunset_unix = weatherIT.sunset_time()
print("The sun rose at",sunrise_unix,"and will set at",sunset_unix)

# Human-readable times
sunrise_date = weatherIT.sunrise_time(timeformat='date')
sunset_date = weatherIT.sunset_time(timeformat='date')
print("The sun rose at",sunrise_date,"and will set at",sunset_date)

The sun rose at 1699856329 and will set at 1699890977
The sun rose at 2023-11-13 06:18:49+00:00 and will set at 2023-11-13 15:56:17+00:00


You can also look for the weather in places whose name are either exactly the one you picked or similar to it.

In [16]:
# Find the weather in Milan
obs_list_acc = mgr.weather_at_places('Milan', 'accurate')

print("Accurate Weather")
for el in obs_list_acc:
    print("The weather in",el.location.name,"is", el.weather.detailed_status)

# Find weather for all the places whose name contains the word "Milan"
obs_list_like = mgr.weather_at_places('Milan', 'like', limit=10)

print("Similar Weather")
for el in obs_list_like:
    print("The weather in",el.location.name,"is", el.weather.detailed_status)

Accurate Weather
The weather in Milan is clear sky
The weather in Milan is overcast clouds
The weather in Milan is clear sky
The weather in Milan is clear sky
The weather in Milan is clear sky
Similar Weather
The weather in Milan is clear sky
The weather in Milan is overcast clouds
The weather in Milan is clear sky
The weather in Milan is clear sky
The weather in Milan is clear sky
The weather in Milán is scattered clouds
The weather in Milan is clear sky
The weather in Milan is few clouds
The weather in Milan is overcast clouds


In [17]:
# Find three hour forecast for 5 days
owm = OWM(api_key)
mgr = owm.weather_manager()
thf = mgr.forecast_at_place('Milan,IT', '3h').forecast

print("These are the forecast from",thf.weathers[0].reference_time('iso'),"to",thf.weathers[len(thf) - 1].reference_time('iso'),"\n")
for weather in thf.weathers:
    print("The forecast at",weather.reference_time('iso'),"in Milan is", weather.status)

These are the forecast from 2023-11-13 12:00:00+00:00 to 2023-11-18 09:00:00+00:00 

The forecast at 2023-11-13 12:00:00+00:00 in Milan is Clear
The forecast at 2023-11-13 15:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-13 18:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-13 21:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 00:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 03:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 06:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 09:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 12:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 15:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 18:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-14 21:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-15 00:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-15 03:00:00+00:00 in Milan is Clouds
The forecast at 2023-11-15 06:00:00+00:00 in Milan is Clouds
T

In [18]:
# Finds forecast for tomorrow at 17 (The outcome is the object with the reference_time closer to the time we are looking for)
from pyowm.utils import timestamps

thf2 = mgr.forecast_at_place('Milan,IT', '3h')
forecast_at_5 = timestamps.tomorrow(17, 0)
weather = thf2.get_weather_at(forecast_at_5)

print("The weather in Milan at",weather.reference_time('iso'),"is", weather.detailed_status)

The weather in Milan at 2023-11-14 18:00:00+00:00 is broken clouds


In [19]:
# Check whether tomorrow will rain
tomorrow = timestamps.tomorrow()
will_it_rain = thf2.will_be_rainy_at(tomorrow)
print("Tomorrow will not rain" if will_it_rain else "Tomorrow will rain")

Tomorrow will rain


In [20]:
# Check when there will be clear and rainy wheather in the following days
when_clear_weather = thf2.when_clear()
when_rain_weather = thf2.when_rain()

if(len(when_clear_weather) > 0):
    print("The weather will be clear at")
    for wcw in when_clear_weather:
        print(wcw.reference_time('iso'))
else:
    print("The weather won't be clear in the next 5 days")

if(len(when_rain_weather) > 0):
    print("The weather will be rainy at")
    for wrw in when_rain_weather:
        print(wrw.reference_time('iso'))
else:
    print("The weather won't be rainy in the next 5 days")

The weather will be clear at
2023-11-13 12:00:00+00:00
2023-11-15 15:00:00+00:00
2023-11-15 18:00:00+00:00
2023-11-15 21:00:00+00:00
2023-11-16 03:00:00+00:00
2023-11-16 06:00:00+00:00
2023-11-16 09:00:00+00:00
2023-11-16 12:00:00+00:00
The weather won't be rainy in the next 5 days


In [21]:
# Check which one of the following days will be the coldest
coldest_weather = thf2.most_cold()
print("The coldest weather in Milan will be at",coldest_weather.reference_time('iso'),"and the weather will be", coldest_weather.detailed_status)

The coldest weather in Milan will be at 2023-11-18 06:00:00+00:00 and the weather will be broken clouds
